In [1]:
# Nativos
import random as rn
import os
import sys
import gc

#calculo
import numpy as np
import pandas as pd
import scipy as sp

#grafico
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display
%matplotlib inline
sns.set(style="whitegrid")

#warning ignore future
import warnings
# warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore")
gc.collect()

BASE_DIR = os.path.dirname(os.getcwd())
if BASE_DIR not in sys.path: sys.path.append(BASE_DIR)

from utils import *
#from graphs import *

SEED = 29082013
os.environ['PYTHONHASHSEED']=str(SEED)
np.random.seed(SEED)
rn.seed(SEED)

subfolder = "data"
os.listdir(subfolder)

['.ipynb_checkpoints',
 'CONTENT_CATEGORY.zip',
 'CONTENT_CATEGORY_BOTTOM.zip',
 'CONTENT_CATEGORY_TOP.zip',
 'conversiones.zip',
 'data.zip',
 'data2.zip',
 'device_data.zip',
 'PAGE.zip',
 'pageviews.zip',
 'pageviews_complemento.zip',
 'pageview_with_previous_2.csv',
 'sampleSubmission.zip',
 'SITE_ID.zip']

## PAGINA SIMPLE

In [2]:
col_user = 'USER_ID'
one_hot_1 = 'PAGE'

In [3]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_1]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
pageview[one_hot_1] = pageview[one_hot_1].apply(lambda _: 'page_' + str(_.split('_')[-1]))
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,PAGE
0,7237,1,page_2
1,4913,1,page_2
2,4913,1,page_2
3,4913,1,page_3
4,4913,1,page_20


In [4]:
cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_1]
).reset_index()
gc.collect()

print(cruce.shape)
cruce.head()

(11676, 1734)


PAGE,USER_ID,page_1,page_10,page_100,page_1000,page_1001,page_1002,page_1003,page_1004,page_1005,...,page_990,page_991,page_992,page_993,page_994,page_995,page_996,page_997,page_998,page_999
0,0,182,737,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,219,271,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,77,142,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,10,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,33,548,3,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [5]:
del pageview
gc.collect()

22

In [6]:
treshold = cruce.shape[0]*1/200
treshold

58.38

In [7]:
cols_del = []
for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print("len(cols_del): ", len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

len(cols_del):  937
(11676, 797)


797

In [8]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_1]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,PAGE
0,7237,1,none_2
1,4913,1,none_2
2,4913,1,2_2
3,4913,1,2_3
4,4913,1,3_20


In [9]:
pageview[one_hot_1] = pageview[one_hot_1].apply(lambda _: 'page_' + str(_.split('_')[-1]))
pageview.head()

,USER_ID,trimestre,PAGE
0,7237,1,page_2
1,4913,1,page_2
2,4913,1,page_2
3,4913,1,page_3
4,4913,1,page_20


In [10]:
trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_1]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

//////////////////////////////////////////////////  trimestre:  1
cols_del:  538
cols_autocompletar:  109
//////////////////////////////////////////////////  trimestre:  2
cols_del:  573
cols_autocompletar:  71
//////////////////////////////////////////////////  trimestre:  3
cols_del:  549
cols_autocompletar:  50
//////////////////////////////////////////////////  trimestre:  4
cols_del:  544
cols_autocompletar:  138


In [11]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

pageview.shape

(44924, 798)

In [12]:
pageview.to_csv('pageview_simple_trimestral.csv', index=False)

In [13]:
try:
    gc.collect()
    del pageview
    gc.collect
except: 
    pass

## PAGINA

In [14]:
col_user = 'USER_ID'
one_hot_1 = 'PAGE'

In [15]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_1]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,PAGE
0,7237,1,none_2
1,4913,1,none_2
2,4913,1,2_2
3,4913,1,2_3
4,4913,1,3_20


In [16]:
pageview.dtypes

USER_ID       int32
trimestre    object
PAGE         object
dtype: object

In [17]:
cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_1]
).reset_index()
gc.collect()

cruce.shape

(11676, 49151)

In [18]:
del pageview
gc.collect()
cruce.head()

PAGE,USER_ID,1000_1,1000_1000,1000_1001,1000_114,1000_12,1000_1227,1000_1231,1000_1252,1000_1254,...,none_875,none_88,none_894,none_9,none_92,none_925,none_93,none_98,none_99,none_993
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,0,0,0,0,0,1,0,0,0,...,0,0,0,0,0,0,0,0,0,0


### Select columns con recurrencia > 0.5%

In [19]:
cols_del = []
for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print("len(cols_del): ", len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

len(cols_del):  45073
(11676, 4078)


4078

### Denuevo cargamos las visitas

In [20]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_1]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,PAGE
0,7237,1,none_2
1,4913,1,none_2
2,4913,1,2_2
3,4913,1,2_3
4,4913,1,3_20


In [21]:
for trimestre in pageview['trimestre'].unique():
    print(
        trimestre,
        pageview[pageview['trimestre'] == trimestre]['USER_ID'].unique().shape
    )

1 (10936,)
2 (11203,)
3 (11356,)
4 (11429,)


In [22]:
trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_1]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

//////////////////////////////////////////////////  trimestre:  1
cols_del:  19376
cols_autocompletar:  518
//////////////////////////////////////////////////  trimestre:  2
cols_del:  18966
cols_autocompletar:  358
//////////////////////////////////////////////////  trimestre:  3
cols_del:  21262
cols_autocompletar:  195
//////////////////////////////////////////////////  trimestre:  4
cols_del:  18874
cols_autocompletar:  734


In [23]:
try:
    gc.collect()
    del cruce
    gc.collect
except: 
    pass

In [24]:
try:
    gc.collect()
    del pageview
    gc.collect
except: 
    pass

In [25]:
len(trimestral)

4

In [26]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

pageview.shape

(44924, 4079)

In [27]:
pageview.to_csv('pageview_trimestral.csv', index=False)

In [28]:
try:
    gc.collect()
    del pageview
    gc.collect
except: 
    pass

In [29]:
try:
    gc.collect()
    del trimestral
    gc.collect
except: 
    pass

## CATEGORIA

In [30]:
one_hot_2 = 'CONTENT_CATEGORY'

In [31]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,CONTENT_CATEGORY
0,7237,1,2_2
1,4913,1,2_2
2,4913,1,2_2
3,4913,1,2_2
4,4913,1,2_2


In [32]:
cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_2]
).reset_index()
gc.collect()

del pageview
gc.collect()
cruce.head()

CONTENT_CATEGORY,USER_ID,10_5,11_4,12_4,13_4,14_5,15_5,16_2,17_2,18_2,...,60_2,61_2,62_2,63_2,64_2,68_6,6_4,7_5,8_5,9_4
0,0,1,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,9,3,1,1
1,1,1,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,3,0,0,0
2,2,11,18,46,32,18,11,0,0,0,...,0,0,0,0,0,0,10,5,3,5
3,3,0,2,0,2,0,0,0,0,0,...,0,1,0,0,0,0,6,0,1,2
4,4,3,1,0,3,3,2,18,8,8,...,0,0,0,0,0,0,15,10,0,5


In [33]:
treshold

58.38

In [34]:
cols_del = []
for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 

In [35]:
len(cols_del)

21

In [36]:
cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

(11676, 45)


45

In [37]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

pageview.head()

(22870354, 3)


,USER_ID,trimestre,CONTENT_CATEGORY
0,7237,1,2_2
1,4913,1,2_2
2,4913,1,2_2
3,4913,1,2_2
4,4913,1,2_2


In [38]:
trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_2]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

//////////////////////////////////////////////////  trimestre:  1
cols_del:  18
cols_autocompletar:  4
//////////////////////////////////////////////////  trimestre:  2
cols_del:  16
cols_autocompletar:  5
//////////////////////////////////////////////////  trimestre:  3
cols_del:  19
cols_autocompletar:  2
//////////////////////////////////////////////////  trimestre:  4
cols_del:  13
cols_autocompletar:  4


In [39]:
try:
    gc.collect()
    del cruce
    gc.collect
except: 
    pass

In [40]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

print(pageview.shape)
pageview.head()

(44924, 46)


CONTENT_CATEGORY,trimestre,USER_ID,10_5,11_4,12_4,13_4,14_5,15_5,16_2,17_2,...,50_5,59_2,5_2,60_2,61_2,63_2,6_4,7_5,8_5,9_4
0,1,0,1,0,0,0,0,0,0,0,...,0,0,10,0,0,0,5,3,1,1
1,1,1,1,0,0,0,0,0,0,0,...,0,0,1,0,0,0,3,0,0,0
2,1,2,1,4,13,8,5,2,0,0,...,0,0,6,0,0,0,4,1,1,0
3,1,3,0,0,0,0,0,0,0,0,...,0,0,4,0,0,0,0,0,0,1
4,1,4,0,0,0,1,0,0,15,7,...,0,0,16,0,0,0,12,7,0,2


In [41]:
pageview.to_csv('content_category_trimestral.csv', index=False)

In [42]:
try:
    gc.collect()
    del pageview
    gc.collect
except: 
    pass

In [43]:
try:
    gc.collect()
    del trimestal
    gc.collect
except: 
    pass

## SITE ID

In [44]:
one_hot_2 = 'SITE_ID'

pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_2]
).reset_index()
gc.collect()

del pageview
gc.collect()
cruce.head()

(22870354, 3)


SITE_ID,USER_ID,1,2,3,4
0,0,202,4090,2139,0
1,1,240,1394,477,0
2,2,402,760,1206,0
3,3,107,813,1013,2
4,4,101,1550,1433,21


In [45]:
treshold

58.38

In [46]:
cols_del = []
for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print(len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

0
(11676, 5)


5

In [47]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_2]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

(22870354, 3)
//////////////////////////////////////////////////  trimestre:  1
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  2
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  3
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  4
cols_del:  0
cols_autocompletar:  0


In [48]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

print(pageview.shape)
pageview.to_csv('site_id_trimestral.csv', index=False)

(44924, 6)


In [49]:
pageview.head()

SITE_ID,trimestre,USER_ID,1,2,3,4
0,1,0,121,1547,173,0
1,1,1,82,483,58,0
2,1,2,103,196,120,0
3,1,3,46,284,16,0
4,1,4,57,820,388,16


### BUSQUEDAS

In [50]:
one_hot_2 = 'ON_SITE_SEARCH_TERM'

pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_2]
).reset_index()
gc.collect()

del pageview
gc.collect()
cruce.head()

(22870354, 3)


ON_SITE_SEARCH_TERM,USER_ID,1,10,100,101,102,103,104,105,106,...,90,91,92,93,94,95,96,97,98,99
0,0,6431,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,1,2111,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,2,2368,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,3,1935,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,4,3105,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [51]:
cols_del = []
print(treshold)


for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print(len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

58.38
293
(11676, 2)


2

In [52]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', one_hot_2]]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
print(pageview.shape)
gc.collect()

trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_2]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

(22870354, 3)
//////////////////////////////////////////////////  trimestre:  1
cols_del:  292
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  2
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  3
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  4
cols_del:  1
cols_autocompletar:  0


In [53]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

print(pageview.shape)
pageview.to_csv('search_trimestral.csv', index=False)

(44924, 3)


In [54]:
pageview.head()

ON_SITE_SEARCH_TERM,trimestre,USER_ID,1
0,1,0,1841
1,1,1,623
2,1,2,419
3,1,3,346
4,1,4,1281


In [55]:
cols_select

['USER_ID', '1']

### DAYS

In [56]:
one_hot_2 = 'tipo_dias'

pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', 'FEC_EVENT']]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
pageview['day'] = pageview['FEC_EVENT'].dt.day
pageview['tipo_dias'] = pageview['day'].apply(get_day_cut)
print(pageview.shape)
gc.collect()

cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_2]
).reset_index()
gc.collect()

del pageview
gc.collect()
cruce.head()

(22870354, 5)


tipo_dias,USER_ID,18_24,7_12,fin_de_mes,inicio_mes,quincena
0,0,1108,1478,843,2071,931
1,1,338,431,374,661,307
2,2,644,597,342,499,286
3,3,292,341,490,679,133
4,4,516,1253,280,541,515


In [57]:
cols_del = []
print(treshold)


for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print(len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

58.38
0
(11676, 6)


6

In [58]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', 'FEC_EVENT']]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
pageview['day'] = pageview['FEC_EVENT'].dt.day
pageview['tipo_dias'] = pageview['day'].apply(get_day_cut)
print(pageview.shape)
gc.collect()

trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_2]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

(22870354, 5)
//////////////////////////////////////////////////  trimestre:  1
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  2
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  3
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  4
cols_del:  0
cols_autocompletar:  0


In [59]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

print(pageview.shape)
pageview.to_csv('rango_dias_trimestral.csv', index=False)

(44924, 7)


In [60]:
pageview.head()

tipo_dias,trimestre,USER_ID,18_24,7_12,fin_de_mes,inicio_mes,quincena
0,1,0,269,421,264,657,230
1,1,1,124,109,142,144,104
2,1,2,94,147,50,51,77
3,1,3,51,58,65,93,79
4,1,4,215,680,96,199,91


### SCHEDULE

In [61]:
one_hot_2 = 'horario'

pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', 'schedule', 'weekday']]


pageview['USER_ID'] = pageview['USER_ID'].astype(int)
pageview[one_hot_2] = pageview['weekday'] + '_' + pageview['schedule']
print(pageview.shape)
gc.collect()

cruce = pd.crosstab(
    index=pageview[col_user], columns=pageview[one_hot_2]
).reset_index()
gc.collect()

del pageview
gc.collect()
cruce.head()

(22870354, 5)


horario,USER_ID,0_almuerzo,0_antes_del_trabajo,0_luego_del_trabajo,0_madrugada,0_trabajo_manana,0_trabajo_tarde,1_almuerzo,1_antes_del_trabajo,1_luego_del_trabajo,...,5_luego_del_trabajo,5_madrugada,5_trabajo_manana,5_trabajo_tarde,6_almuerzo,6_antes_del_trabajo,6_luego_del_trabajo,6_madrugada,6_trabajo_manana,6_trabajo_tarde
0,0,170,43,149,0,568,43,175,57,243,...,137,0,121,201,32,6,48,0,68,69
1,1,122,10,85,24,189,12,30,18,100,...,13,0,27,17,54,0,19,0,27,21
2,2,284,0,44,15,102,40,251,20,10,...,14,19,9,7,0,0,32,0,4,61
3,3,139,0,57,0,109,102,19,0,86,...,43,8,26,60,53,6,26,0,55,73
4,4,226,72,89,83,193,45,41,32,42,...,26,8,45,32,4,0,12,23,68,8


In [62]:
cols_del = []
print(treshold)


for col in list(cruce.columns)[1:]:
    
    columna = cruce[col]
    size = len(columna[columna > 0])
    
    if size < treshold:
        cols_del.append(col) 
        
print(len(cols_del))

cruce = cruce.drop(cols_del, axis=1)
print(cruce.shape)

cols_select = list(cruce.columns)
del cruce
gc.collect()
len(cols_select)

58.38
0
(11676, 43)


43

In [63]:
pageview = pd.read_csv(
    "{}/pageview_with_previous_2.csv".format(subfolder), parse_dates=["FEC_EVENT"], dtype=str
).sort_values(
    ["FEC_EVENT"]
)[[col_user, 'trimestre', 'schedule', 'weekday']]

pageview['USER_ID'] = pageview['USER_ID'].astype(int)
pageview[one_hot_2] = pageview['weekday'] + '_' + pageview['schedule']
print(pageview.shape)
gc.collect()

trimestral = []

for trimestre in pageview['trimestre'].unique():
    print("/"*50, " trimestre: ", trimestre)
    cruce = pd.crosstab(
        pageview[pageview['trimestre'] == trimestre][col_user], 
        pageview[pageview['trimestre'] == trimestre][one_hot_2]
    ).reset_index()
    
    cols_cruce = list(cruce.columns)
    cols_del = set(cols_cruce) - set(cols_select)
    print("cols_del: ", len(cols_del))
    
    cols_autocompletar = set(cols_select) - set(cols_cruce)
    print("cols_autocompletar: ", len(cols_autocompletar))
    for col in cols_autocompletar:
        cruce[col] = 0
    
    cruce['trimestre'] = trimestre
    
    trimestral.append(cruce[['trimestre'] + cols_select])
    del cols_cruce
    del cruce
    
    pageview = pageview[pageview['trimestre'] != trimestre]
    gc.collect()

(22870354, 5)
//////////////////////////////////////////////////  trimestre:  1
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  2
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  3
cols_del:  0
cols_autocompletar:  0
//////////////////////////////////////////////////  trimestre:  4
cols_del:  0
cols_autocompletar:  0


In [64]:
pageview = pd.concat(
    trimestral,
    ignore_index=True,
    axis=0
)

print(pageview.shape)
pageview.to_csv('schedule_trimestral.csv', index=False)

(44924, 44)


In [65]:
pageview.head()

horario,trimestre,USER_ID,0_almuerzo,0_antes_del_trabajo,0_luego_del_trabajo,0_madrugada,0_trabajo_manana,0_trabajo_tarde,1_almuerzo,1_antes_del_trabajo,...,5_luego_del_trabajo,5_madrugada,5_trabajo_manana,5_trabajo_tarde,6_almuerzo,6_antes_del_trabajo,6_luego_del_trabajo,6_madrugada,6_trabajo_manana,6_trabajo_tarde
0,1,0,41,25,58,0,179,17,46,5,...,23,0,28,36,6,0,0,0,12,25
1,1,1,53,0,0,0,38,0,8,0,...,13,0,19,0,11,0,12,0,22,0
2,1,2,93,0,4,0,21,0,31,0,...,0,0,6,0,0,0,0,0,4,0
3,1,3,30,0,2,0,16,32,10,0,...,0,8,4,18,2,0,4,0,11,5
4,1,4,118,40,23,57,41,11,17,11,...,24,7,4,3,0,0,5,22,10,8


In [66]:
try:
    gc.collect()
    del pageview
except:
    pass

In [67]:
try:
    gc.collect()
    del cruce
except:
    pass

In [68]:
try:
    gc.collect()
    del trimestral
except:
    pass